# Importing necessary libraries

In [1]:
#import necessary dependecies
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import os
import warnings
import numpy as np  
import seaborn as sns
import pandas as pd, os, gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
%matplotlib inline
warnings.filterwarnings('ignore')
from typing import List

C:\Users\SHOLAY\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Environment setup

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# Dataset

In [3]:
trains = 'Train.csv'
tests = 'Test.csv'
submissions = 'SampleSubmission.csv'

# Function to load data

In [4]:
def read_data(data, parse_date=None):
    if parse_date is not None:
        return pd.read_csv(data, parse_dates=[parse_date])
    else:
        return pd.read_csv(data)

# Loading the datasets

In [5]:
train = read_data(trains, parse_date='join_date')
test = read_data(tests, parse_date='join_date')
submission= read_data(submissions)

# Data overview

In [6]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,2019-01-02,F,M,1987,1X1H,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,2019-01-06,F,M,1981,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,2013-01-06,M,U,1991,748L,QZYX,90QI,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,2019-01-08,M,M,1990,1X1H,BP09,56SI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,2019-01-08,M,M,1990,748L,NO3L,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [7]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,2018-01-12,M,M,1984,94KC,DZRV,90QI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H6141K3,2019-01-10,M,M,1996,1X1H,J9SY,90QI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,2020-01-01,F,W,1968,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,2019-01-02,M,M,1989,94KC,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,2020-01-02,F,M,1982,UAOD,0KID,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29132 entries, 0 to 29131
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ID                        29132 non-null  object        
 1   join_date                 29130 non-null  datetime64[ns]
 2   sex                       29132 non-null  object        
 3   marital_status            29132 non-null  object        
 4   birth_year                29132 non-null  int64         
 5   branch_code               29132 non-null  object        
 6   occupation_code           29132 non-null  object        
 7   occupation_category_code  29132 non-null  object        
 8   P5DA                      29132 non-null  int64         
 9   RIBP                      29132 non-null  int64         
 10  8NN1                      29132 non-null  int64         
 11  7POT                      29132 non-null  int64         
 12  66FJ              

# Checking the data

In [9]:
print(f'Total columns for training set: {len(train.columns)}');print(f'Total columns for testing set: {len(train.columns)}')

Total columns for training set: 29
Total columns for testing set: 29


In [10]:
train.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [11]:
#print all dataset shape
print(f'Train shape: {train.shape}\nTest shape:  {test.shape}\nSubmission shape: {submission.shape}')

Train shape: (29132, 29)
Test shape:  (10000, 29)
Submission shape: (210000, 2)


In [12]:
#Store Products ID in Multidimensional array
products = train[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
products_t = test[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]

In [13]:
#Axis Transformation
train = train.melt(id_vars=train.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )

In [14]:
#A merger
train['combiner']='x'

In [15]:
#Set
data=pd.concat([train,test],sort=False).reset_index(drop=True)

In [16]:
#preview random 3 sample
data.sample(3)

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,Label,combiner
550424,UKG8FLC,2019-01-04,M,M,1982,94KC,SST3,56SI,J9JW,0,x
670514,ER2VUID,2019-01-01,M,M,1993,UAOD,UJ5T,T4MS,GYSR,0,NaN
817453,1B8FK7X,2019-01-04,M,M,1995,O67J,UJ5T,T4MS,ECY3,0,NaN


In [17]:
#Confirm Distribution (TEST AND SAMPLE SUBMISSION FILE)
print('==' * 18);print('TRAIN LABEL DISTRIBUTION');print('==' * 18);print(train['Label'].value_counts())
print('==' * 18);print('TEST LABEL DISTRIBUTION');print('==' * 18);print(test['Label'].value_counts())
print('==' * 18);print('SAMPLE SUBMISSION LABEL DISTRIBUTION');print('==' * 18);print(submission['Label'].value_counts())

TRAIN LABEL DISTRIBUTION
0    545419
1     66353
Name: Label, dtype: int64
TEST LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64
SAMPLE SUBMISSION LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64


In [18]:
train=data[data.combiner.notnull()].reset_index(drop=True);test=data[data.combiner.isna()].reset_index(drop=True)
train.drop('combiner', inplace=True, axis=1);test.drop(['Label','combiner'], inplace=True, axis=1)
print(train.shape, test.shape, submission.shape)

(611772, 10) (210000, 9) (210000, 2)


In [19]:
#Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE'];test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

In [20]:
#Re-arrange column (not necessary)
train=train[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE',  'Label']]
test=test[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE']]

In [21]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,4WKQSBB,2019-01-02,F,M,1987,1X1H,2A7I,T4MS,P5DA,4WKQSBB X P5DA,0
1,CP5S02H,2019-01-06,F,M,1981,UAOD,2A7I,T4MS,P5DA,CP5S02H X P5DA,0
2,2YKDILJ,2013-01-06,M,U,1991,748L,QZYX,90QI,P5DA,2YKDILJ X P5DA,0
3,2S9E81J,2019-01-08,M,M,1990,1X1H,BP09,56SI,P5DA,2S9E81J X P5DA,0
4,BHDYVFT,2019-01-08,M,M,1990,748L,NO3L,T4MS,P5DA,BHDYVFT X P5DA,0


In [22]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,F86J5PC,2018-01-12,M,M,1984,94KC,DZRV,90QI,P5DA,F86J5PC X P5DA
1,H6141K3,2019-01-10,M,M,1996,1X1H,J9SY,90QI,P5DA,H6141K3 X P5DA
2,RBAYUXZ,2020-01-01,F,W,1968,UAOD,2A7I,T4MS,P5DA,RBAYUXZ X P5DA
3,KCBILBQ,2019-01-02,M,M,1989,94KC,2A7I,T4MS,P5DA,KCBILBQ X P5DA
4,LSEC1ZJ,2020-01-02,F,M,1982,UAOD,0KID,T4MS,P5DA,LSEC1ZJ X P5DA


In [23]:
#Lets Check for data co-existence in Train/Test
for col in ['PCODE','occupation_code', 'branch_code','occupation_category_code']:
    train_col = train[col].unique();test_col = test[col].unique()
    print ('Not in Test ' + col + ' ',[i  for i in train_col if i not in test_col])
    print ('Not in Train ' + col + ' ',[i  for i in test_col  if i not in train_col]);print('==' * 30)

Not in Test PCODE  []
Not in Train PCODE  []
Not in Test occupation_code  ['IE90', 'Q0LY', 'JSAX', '6XXU', '8HRZ', 'OQMY', 'INEJ', 'VZN9', 'UC7E', 'PSUY', 'WSRG', 'JQH3', 'LGTN', '738L', 'QQUP', '2XZ1', 'CAAV', 'LLLH', 'W1X2', 'DHSN', 'IX8T', '2US6', 'ZWPL', 'MEFQ', '9B5B', 'JUIP', 'BFD1', 'A4ZC', 'IMHI', 'E5PF', 'GZA8', '3YQ1', 'PJR4', 'NDL9', 'PPNK', '2686', '5LNN', '374O', 'URYD', 'M0WG', 'KBWO', 'ONY7', 'VYSA', 'KUPK', 'R7GL', 'HSVE', 'BER4', '6SKY', 'RH2K', 'ZHC2', 'W3ZV', 'FLXH', 'UYDZ', 'YJXM', '59QM']
Not in Train occupation_code  ['0ZND', '8CHJ', '9F96', 'HSI5', '93OJ', 'BIA0', 'E2MJ', 'JBJP', '0FOI']
Not in Test branch_code  []
Not in Train branch_code  []
Not in Test occupation_category_code  []
Not in Train occupation_category_code  []


In [24]:
train.tail()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
611767,8SBA93U,2019-01-09,F,M,1974,748L,2A7I,T4MS,ECY3,8SBA93U X ECY3,0
611768,Y73ETTO,2018-01-11,M,M,1987,XX25,Q6J6,90QI,ECY3,Y73ETTO X ECY3,0
611769,MVXG2LC,2019-01-02,M,M,1985,BOAS,6PE7,T4MS,ECY3,MVXG2LC X ECY3,0
611770,BUQ8DQN,2019-01-07,M,M,1989,UAOD,2A7I,T4MS,ECY3,BUQ8DQN X ECY3,0
611771,DEJBWYU,2020-01-03,M,M,1975,94KC,31JW,90QI,ECY3,DEJBWYU X ECY3,0


In [25]:
test.tail()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
209995,0GMU5UH,2019-01-04,F,M,1986,748L,2A7I,T4MS,ECY3,0GMU5UH X ECY3
209996,I9W11CD,2019-01-08,M,M,1980,UAOD,0KID,T4MS,ECY3,I9W11CD X ECY3
209997,42WTEGT,2018-01-04,M,M,1993,748L,SST3,56SI,ECY3,42WTEGT X ECY3
209998,8EKC4O9,2015-01-05,F,U,1981,49BM,2A7I,T4MS,ECY3,8EKC4O9 X ECY3
209999,WHII93K,2018-01-11,M,U,1977,94KC,2A7I,T4MS,ECY3,WHII93K X ECY3


In [26]:
# LABEL ENCODE
def encode_LE(train,test,cols,verbose=True):
    for col in cols:
        df_comb = pd.concat([train[col],test[col]],axis=0)
        df_comb,_ = df_comb.factorize(sort=True)
        nm = col
        if df_comb.max()>32000: 
            train[nm] = df_comb[:len(train)].astype('int32')
            test[nm] = df_comb[len(train):].astype('int32')
        else:
            train[nm] = df_comb[:len(train)].astype('int16')
            test[nm] = df_comb[len(train):].astype('int16')
        del df_comb; x=gc.collect()
        if verbose: print(nm,', ',end='')

In [27]:
encode_LE(train, test, ['ID','branch_code', 'occupation_code','occupation_category_code','PCODE','sex','marital_status'])

ID , branch_code , occupation_code , occupation_category_code , PCODE , sex , marital_status , 

In [28]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,5384,2019-01-02,0,2,1987,0,19,5,15,4WKQSBB X P5DA,0
1,13834,2019-01-06,0,2,1981,11,19,5,15,CP5S02H X P5DA,0
2,3217,2013-01-06,1,6,1991,3,185,1,15,2YKDILJ X P5DA,0
3,3010,2019-01-08,1,2,1990,0,82,0,15,2S9E81J X P5DA,0
4,12546,2019-01-08,1,2,1990,3,157,5,15,BHDYVFT X P5DA,0


In [29]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,16566,2018-01-12,1,2,1984,4,96,1,15,F86J5PC X P5DA
1,18664,2019-01-10,1,2,1996,0,128,1,15,H6141K3 X P5DA
2,29677,2020-01-01,0,7,1968,11,19,5,15,RBAYUXZ X P5DA
3,22120,2019-01-02,1,2,1989,4,19,5,15,KCBILBQ X P5DA
4,23700,2020-01-02,0,2,1982,11,3,5,15,LSEC1ZJ X P5DA


In [30]:
def insert_year(meth):
    meth.fillna(method='pad', inplace=True)
    meth.insert(loc= 2, column= 'year', value=meth['join_date'].apply(lambda x: x.year))

def insert_month(meth):
    meth.fillna(method='pad', inplace=True)
    meth.insert(loc= 3, column= 'month', value=meth['join_date'].apply(lambda x: x.month))
    
def insert_day(meth):
    meth.fillna(method='pad', inplace=True)
    meth.insert(loc= 2, column= 'day', value=meth['join_date'].apply(lambda x: x.day))

In [31]:
hold_data= [train, test]
for i in hold_data:
    insert_year(i)
    insert_month(i)
    insert_day(i)

In [32]:
test.head()

,ID,join_date,day,year,month,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,16566,2018-01-12,12,2018,1,1,2,1984,4,96,1,15,F86J5PC X P5DA
1,18664,2019-01-10,10,2019,1,1,2,1996,0,128,1,15,H6141K3 X P5DA
2,29677,2020-01-01,1,2020,1,0,7,1968,11,19,5,15,RBAYUXZ X P5DA
3,22120,2019-01-02,2,2019,1,1,2,1989,4,19,5,15,KCBILBQ X P5DA
4,23700,2020-01-02,2,2020,1,0,2,1982,11,3,5,15,LSEC1ZJ X P5DA


# MODEL TRAINING

In [33]:
for date in hold_data:
    date.drop(columns=['join_date'], inplace=True)

In [34]:
train.head(2)

,ID,day,year,month,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,5384,2,2019,1,0,2,1987,0,19,5,15,4WKQSBB X P5DA,0
1,13834,6,2019,1,0,2,1981,11,19,5,15,CP5S02H X P5DA,0


In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns=['ID X PCODE', 'Label']), train['Label'], test_size=0.2, random_state=42)

In [37]:
X_train.head()

,ID,day,year,month,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE
517604,28656,11,2017,1,1,2,1986,7,3,5,11
47532,11453,6,2019,1,0,2,1984,13,164,5,18
369762,30498,2,2017,1,1,2,1985,3,140,1,4
61472,32907,9,2019,1,1,2,1990,1,222,1,2
556184,14310,9,2017,1,1,2,1997,3,19,5,7


# Using Random forest classifier

In [45]:
rd = RandomForestClassifier(n_estimators=20)

In [46]:
rd.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [47]:
rd.score(X_test, y_test)

0.9576069633443668

In [48]:
test.head(2)

,ID,day,year,month,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,16566,12,2018,1,1,2,1984,4,96,1,15,F86J5PC X P5DA
1,18664,10,2019,1,1,2,1996,0,128,1,15,H6141K3 X P5DA


In [51]:
data= pd.DataFrame(rd.predict(test.drop(columns=['ID X PCODE'])))

In [55]:
data.insert(loc=0, column='ID X PCODE', value=test['ID X PCODE'])

In [61]:
data.columns= ['ID X PCODE', 'Label']
data.to_csv('Prediction.csv', index= False)

In [62]:
data.shape

(210000, 2)

In [64]:
data['Label'].value_counts()

0    190761
1     19239
Name: Label, dtype: int64

In [66]:
submission['Label'].value_counts()

0    197147
1     12853
Name: Label, dtype: int64

# Using Logistic regression

In [41]:
from sklearn.linear_model import LogisticRegression

In [42]:
log = LogisticRegression()

In [43]:
log.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
log.score(X_test, y_test)

0.8912835601324016